Initialization

In [61]:
from scipy.optimize import minimize
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp,Operator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator
from qiskit.quantum_info import Statevector,random_statevector
import math
import json
import os
import random

GREEN = "\033[92m"
RED = "\033[91m"
YELLOW = "\033[93m"
BLUE = "\033[94m"
CYAN = "\033[96m"
MAGENTA = "\033[95m"
BOLD = "\033[1m"
UNDERLINE = "\033[4m"
STANDARD = "\033[0m"

%run config.py

Load the directories and identify the project inside them.

In [62]:
def rotfitness(parameters, *args):
	qbits=int(len(parameters)/3)

	qc = QuantumCircuit(qbits)
	for i in range(qbits):
		qc.rx(float(parameters[3*i]), i)
		qc.ry(float(parameters[3*i+1]), i)
		qc.rz(float(parameters[3*i+2]), i)
	
	# print (qc.draw())
	qc=qc.reverse_bits()

	complexArray = []
	complexArray.append(complex(1,0))
	for i in range(2**qbits-1):
		complexArray.append(complex(0,0))

	sv = Statevector(complexArray)
	sv = sv.evolve(qc)

	outlistcomplex = sv.data.tolist()
	# print(outlistcomplex)
	# print (args)

	sigma = 0.0
	for i in range(len(outlistcomplex)):
		sigma  += abs( args[i].real - outlistcomplex[i].real)
		sigma  += abs( args[i].imag - outlistcomplex[i].imag)

	sigma = sigma/len(outlistcomplex)
	# print (sigma)
	return sigma

In [63]:
def guessrotations(target):
	# target is a statevector
	length = len(target)
	qbits = int(math.log2(length))
	# print (qbits)
	initial_guess = [1.0,] * 3*qbits
	# print(initial_guess)
	result = minimize(rotfitness, initial_guess, args=tuple(target))
	return result.x.tolist()

In [64]:
# List all the directories, only the directories
dirs = [d for d in os.listdir('.') if os.path.isdir(d)]
activeproj = {}
base = ""
for d in dirs:
	if os.path.exists(d+"/active"):
		with open(d+"/active","r") as f:
			ptype = f.read().strip()
			activeproj[d] = ptype
		if os.path.exists(d+"/base"):
			base = d
			if ptype != "qiskit":
				print("Warning: base project is not of type qiskit")
		
# print("Active projects: ",activeproj)
# print("Static data: ",staticdata)
# print("Base project: ",base)

In [65]:
def getRandomStateVectorWhole(qbits):
	sv=random_statevector(2**qbits)
	rot=guessrotations(sv)
	return sv,rot

In [66]:
def getRandomStateVectorRotations(qbits):
	states=[]
	rots=[]
	for i in range(qbits):
		# Random rotation 0 4pi
		theta = 2*math.pi*random.random()
		phi = 2 *math.pi*random.random()

		rot=[theta,phi,0]

		qc = QuantumCircuit(1)
		qc.rx(theta, 0)
		qc.ry(phi, 0)
	
		complexArray = []
		complexArray.append(complex(1,0))
		complexArray.append(complex(0,0))

		sv = Statevector(complexArray)
		sv = sv.evolve(qc)

		states.append(sv)
		rots.append(rot)

	rot=[]
	for i in range(qbits):
		rot+=rots[i]

	sv=states[0]
	for i in range(1,qbits):
		newsv=[]
		for j in range(len(sv.data)):
			for k in range(2):
				newsv.append(sv.data[j]*states[i].data[k])
		sv=Statevector(newsv)

	print(sv)
	return sv,rot

In [67]:
def getRandomStateVectorSingle(qbits):
	states=[]
	rots=[]
	for i in range(qbits):
		sv=random_statevector(2**qbits)
		# sv= Statevector([1/math.sqrt(2), 1j/math.sqrt(2)])
		# sv= Statevector([0, 1])
		states.append(sv)
		print ("State: ",sv)
		ai = sv.data[0].imag
		ar = sv.data[0].real
		bi = sv.data[1].imag
		br = sv.data[1].real
		d2 = br**2+ai**2
		print ("d2: ",d2)
		c2 = ai**2/(d2)
		print ("c2: ",c2)
		phi = 2.0*math.asin(math.sqrt(d2))
		print ("phi: ",phi)
		theta = 2.0* math.asin(math.sqrt(c2))
		print ("theta: ",theta)
		print ("rx:")
		print ( math.cos(theta/2), "-i",math.sin(theta/2))
		print ( "-i", math.sin(theta/2), math.cos(theta/2))

		print ("ry:")
		print ( math.cos(phi/2), "-",math.sin(phi/2))
		print ( math.sin(phi/2), math.cos(phi/2))

		g0ar = math.cos(theta/2)
		g0bi = math.sin(theta/2)
		print ("g0 = rx * 0")
		print (g0ar, "-i",g0bi)

		print ("g1 = ry * g0")
		

		rot = [theta,phi,0]
	# sv=random_statevector(2**qbits)
	# rot=guessrotations(sv)
	return sv,rot

# getRandomStateVector(2)

Create a set of inputs for the test cases. They should be normalized values, let's use the qiskit random_statevector function to generate them.

In [68]:
if fundamental:
	inputs = []
	rots = []
	outlist=[[1.0,0.0]]
	rotlist=[0.0,] * 3 * qbits
	for i in range(2**qbits-1):
		outlist.append([0.0,0.0])
	inputs.append(outlist)
	rots.append(rotlist)

	with open('inputs.json', 'w') as f:
		json.dump(inputs, f)

	with open('rots.json', 'w') as f:
		json.dump(rots, f)
else:
	if staticdata:
		with open('inputs.json', 'r') as f:
			inputs = json.load(f)
		with open('rots.json', 'r') as f:
			rots = json.load(f)
	else:
		inputs = []
		rots = []
		for i in range(numiter):
			sv,rot=getRandomStateVectorRotations(qbits)
			outlist=[]
			for i in range(len(sv)):
				outlist.append([sv[i].real,sv[i].imag])
			rots.append(rot)
			inputs.append(outlist)


		with open('inputs.json', 'w') as f:
			json.dump(inputs, f)

		with open('rots.json', 'w') as f:
			json.dump(rots, f)

Statevector([-0.30397512+0.30748286j,  0.24185782+0.21522484j,
             -0.18399388-0.64803288j, -0.47734457+0.16302846j],
            dims=(2, 2))
Statevector([-0.08497096+0.01864457j, -0.3981599 +0.47471036j,
              0.05936181+0.09077988j,  0.72962522+0.25383719j],
            dims=(2, 2))
Statevector([-0.620655  +0.07281515j,  0.57031464-0.11451099j,
              0.162703  +0.34464779j, -0.17702476-0.30744372j],
            dims=(2, 2))
Statevector([-0.19054167-0.74061318j,  0.15311452-0.4148186j ,
             -0.33246409+0.23258474j, -0.23460188+0.00118006j],
            dims=(2, 2))
Statevector([ 0.07459437-0.25355642j, -0.40536394+0.11624324j,
              0.37209981+0.27150677j,  0.0477139 -0.73339048j],
            dims=(2, 2))
Statevector([-0.19204111+0.0864428j ,  0.26970705-0.20506163j,
              0.46876191+0.12083663j, -0.77854487-0.01960521j],
            dims=(2, 2))
Statevector([-0.88300274+0.0175588j , -0.14563674-0.33369899j,
              0.17272281-

Convert the inputs to the format expected by the single project.

In [69]:
for p in activeproj:
	if activeproj[p] == "qiskit" or activeproj[p] == "pennylane" or activeproj[p] == "spinqsim" or activeproj[p] == "spinq":
		inputsJson = []
		for i in inputs:
			inputJson=[]
			for j in i:
				val= [j[0], j[1]]
				inputJson.append(val)
			inputsJson.append( inputJson )

		with open(p+'/inputs.json', 'w') as f:
			json.dump(inputsJson, f)
		with open(p+'/rots.json', 'w') as f:
			json.dump(rots, f)

	if activeproj[p] == "bmqsim":
		inputsJson = []
		for i in inputs:
			inputJson=[]
			for j in i:
				val= {"Real": j[0], "Imag": j[1]}
				inputJson.append(val)
			inputsJson.append({ "Vector": inputJson })

		with open(p+'/inputs.json', 'w') as f:
			json.dump(inputsJson, f)
		with open(p+'/rots.json', 'w') as f:
			json.dump(rots, f)

	if activeproj[p] == "bmqsim_hw" or activeproj[p] == "quest":
		inputValues=[]
		for i in inputs:
			for j in i:
				inputValues.append(j[0])
				inputValues.append(j[1])

		with open(p+'/inputs.txt', 'w') as f:
			for item in inputValues:
				f.write("%s\n" % item)
		with open(p+'/rots.json', 'w') as f:
			json.dump(rots, f)
			

Run all the projects

In [70]:
for p in activeproj:	
	cwd = os.getcwd()
	os.chdir(p)
	if os.path.exists("circuit.py"):
		if activeproj[p] == "spinqsim" or activeproj[p] == "spinq":
			! source /tools/Conda/etc/profile.d/conda.sh && conda activate spinq && python circuit.py && conda deactivate
		elif activeproj[p] == "pennylane":
			! source /tools/Conda/etc/profile.d/conda.sh && conda activate pennylane && python circuit.py && conda deactivate
		else:
			%run circuit.py
	elif os.path.exists("circuit.ipynb"):
		%run circuit.ipynb
	os.chdir(cwd)

     ┌────────────┐┌────────────┐     
q_0: ┤ Rx(1.8608) ├┤ Ry(2.9647) ├─────
     └┬──────────┬┘├────────────┤┌───┐
q_1: ─┤ Rx(5.37) ├─┤ Ry(3.9628) ├┤ X ├
      └──────────┘ └────────────┘└───┘
     ┌─────────────┐┌────────────┐     
q_0: ┤ Rx(0.16896) ├┤ Ry(2.9185) ├─────
     └┬────────────┤├───────────┬┘┌───┐
q_1: ─┤ Rx(1.1493) ├┤ Ry(4.153) ├─┤ X ├
      └────────────┘└───────────┘ └───┘
     ┌────────────┐┌────────────┐     
q_0: ┤ Rx(3.2227) ├┤ Ry(1.6426) ├─────
     ├────────────┤├────────────┤┌───┐
q_1: ┤ Rx(2.1393) ├┤ Ry(2.5874) ├┤ X ├
     └────────────┘└────────────┘└───┘
      ┌────────────┐┌────────────┐     
q_0: ─┤ Rx(3.6574) ├┤ Ry(4.1016) ├─────
     ┌┴────────────┤├────────────┤┌───┐
q_1: ┤ Rx(0.88729) ├┤ Ry(5.8049) ├┤ X ├
     └─────────────┘└────────────┘└───┘
     ┌─────────────┐ ┌────────────┐     
q_0: ┤ Rx(0.86319) ├─┤ Ry(3.9773) ├─────
     └┬────────────┤┌┴────────────┤┌───┐
q_1: ─┤ Rx(4.0903) ├┤ Ry(0.52362) ├┤ X ├
      └────────────┘└─────────────┘└───┘
     

OUTPUT checks the output of the projects and compares them with the pivot project.


In [71]:
passed = True
passwdProjects = {}
for p in activeproj:
	if (activeproj[p] == "qiskit" and p!=base) or activeproj[p] == "spinqsim" or activeproj[p] == "pennylane":
		with open(base+'/outputs.json', 'r') as f:
			qiskitoutputs = json.load(f)

		with open(p+'/outputs.json', 'r') as f:
			probs = json.load(f)

		if len(qiskitoutputs) != len(probs):
			print("Warning: Different number of outputs")

		qvars = []

		for i in range(len(qiskitoutputs[0])):
			qvars.append([0,0])

		for i in range(len(qiskitoutputs)):
			qiskitoutput = qiskitoutputs[i]
			spinqoutput = probs[i]
			for j in range(len(qiskitoutput)):
				qreal = float(qiskitoutput[j][0])
				qimag = float(qiskitoutput[j][1])
				qreal2 = float(spinqoutput[j][0])
				qimag2 = float(spinqoutput[j][1])
				if useprob:
					qvars[j][0]=pow(pow(qreal,2)+pow(qimag,2)-pow(qreal2,2)-pow(qimag2,2),2)
				else:
					qvars[j][0] += pow(qreal - qreal2, 2)
					qvars[j][1] += pow(qimag - qimag2, 2)

		for i in range(len(qvars)):
			qvars[i][0] = pow(qvars[i][0]/len(qiskitoutputs), 0.5)
			qvars[i][1] = pow(qvars[i][1]/len(qiskitoutputs), 0.5)
			if qvars[i][0] > passthreshold or qvars[i][1] > passthreshold:
				passed = False
				passwdProjects[p] = False

		if not quiet:
			print("Average error on "+p+":")
			print(json.dumps(qvars, indent=4))
		
		if p not in passwdProjects:
			passwdProjects[p] = True

	if activeproj[p] == "spinq":
		if not useprob:
			panic("spinq only supports useprob mode")

		with open(base+'/outputs.json', 'r') as f:
			qiskitoutputs = json.load(f)

		with open(p+'/probs.json', 'r') as f:
			probs = json.load(f)

		if len(qiskitoutputs) != len(probs):
			print("Warning: Different number of outputs")

		qvars = []

		for i in range(len(qiskitoutputs[0])):
			qvars.append([0,0])

		for i in range(len(qiskitoutputs)):
			qiskitoutput = qiskitoutputs[i]
			spinqoutput = probs[i]
			for j in range(len(qiskitoutput)):
				qreal = float(qiskitoutput[j][0])
				qimag = float(qiskitoutput[j][1])
				prob = float(spinqoutput[j])
				if useprob:
					qvars[j][0]=pow(pow(qreal,2)+pow(qimag,2)-prob,2)

		for i in range(len(qvars)):
			qvars[i][0] = pow(qvars[i][0]/len(qiskitoutputs), 0.5)
			qvars[i][1] = pow(qvars[i][1]/len(qiskitoutputs), 0.5)
			if qvars[i][0] > passthreshold or qvars[i][1] > passthreshold:
				passed = False
				passwdProjects[p] = False

		if not quiet:
			print("Average error on "+p+":")
			print(json.dumps(qvars, indent=4))

		if p not in passwdProjects:
			passwdProjects[p] = True
			
	if activeproj[p] == "bmqsim":
		
		with open(base+'/outputs.json', 'r') as f:
			qiskitoutputs = json.load(f)

		with open(p+'/outputs.json', 'r') as f:
			bmqsimoutputs = json.load(f)

		if len(qiskitoutputs) != len(bmqsimoutputs):
			print("Warning: Different number of outputs")

		bmvars = []

		for i in range(len(qiskitoutputs[0])):
			bmvars.append([0,0])

		for i in range(len(qiskitoutputs)):
			qiskitoutput = qiskitoutputs[i]
			bmqsimoutput = bmqsimoutputs[i]["Vector"]
			for j in range(len(qiskitoutput)):
				qreal = float(qiskitoutput[j][0])
				qimag = float(qiskitoutput[j][1])
				breal = float(bmqsimoutput[j]["Real"])
				bimag = float(bmqsimoutput[j]["Imag"])
				if useprob:
					bmvars[j][0]=pow(pow(qreal,2)+pow(qimag,2)-pow(breal,2)-pow(bimag,2),2)
				else:
					bmvars[j][0] += pow(qreal - breal, 2)
					bmvars[j][1] += pow(qimag - bimag, 2)

		for i in range(len(bmvars)):
			bmvars[i][0] = pow(bmvars[i][0]/len(qiskitoutputs), 0.5)
			bmvars[i][1] = pow(bmvars[i][1]/len(qiskitoutputs), 0.5)
			if bmvars[i][0] > passthreshold or bmvars[i][1] > passthreshold:
				passed = False
				passwdProjects[p] = False

		if not quiet:
			print("Average error on "+p+":")
			print(json.dumps(bmvars, indent=4))

		if p not in passwdProjects:
			passwdProjects[p] = True

	if activeproj[p] == "bmqsim_hw" or activeproj[p] == "quest":
		with open(base+'/outputs.json', 'r') as f:
			qiskitoutputs = json.load(f)

		with open(p+'/outputs.txt', 'r') as f:
			bmqsimoutputs = []
			for k in range(len(qiskitoutputs)):
				bmqsimoutput=[]
				for i in range(2 ** qbits):
					line = f.readline()
					vreal=float(line)
					line = f.readline()
					vimm=float(line)
					bmqsimoutput.append([vreal,vimm])
				bmqsimoutputs.append(bmqsimoutput)

		if len(qiskitoutputs) != len(bmqsimoutputs):
			print("Warning: Different number of outputs")

		bmvars = []

		for i in range(len(qiskitoutputs[0])):
			bmvars.append([0,0])

		for i in range(len(qiskitoutputs)):
			qiskitoutput = qiskitoutputs[i]
			bmqsimoutput = bmqsimoutputs[i]
			for j in range(len(qiskitoutput)):
				qreal = float(qiskitoutput[j][0])
				qimag = float(qiskitoutput[j][1])
				breal = float(bmqsimoutput[j][0])
				bimag = float(bmqsimoutput[j][1])
				if useprob:
					bmvars[j][0]=pow(pow(qreal,2)+pow(qimag,2)-pow(breal,2)-pow(bimag,2),2)
				else:
					bmvars[j][0] += pow(qreal - breal, 2)
					bmvars[j][1] += pow(qimag - bimag, 2)

		for i in range(len(bmvars)):
			bmvars[i][0] = pow(bmvars[i][0]/len(qiskitoutputs), 0.5)
			bmvars[i][1] = pow(bmvars[i][1]/len(qiskitoutputs), 0.5)
			if bmvars[i][0] > passthreshold or bmvars[i][1] > passthreshold:
				passed = False
				passwdProjects[p] = False

		if not quiet:
			print("Average error on "+p+":")
			print(json.dumps(bmvars, indent=4))

		if p not in passwdProjects:
			passwdProjects[p] = True

Average error on pennylane:
[
    [
        0.0,
        0.0
    ],
    [
        4.388541835720876e-18,
        0.0
    ],
    [
        2.7428386473255477e-19,
        0.0
    ],
    [
        0.0,
        0.0
    ]
]
Average error on qiskitrot:
[
    [
        0.0,
        0.0
    ],
    [
        3.949687652148789e-17,
        0.0
    ],
    [
        8.228515941976643e-19,
        0.0
    ],
    [
        0.0,
        0.0
    ]
]
Average error on quest:
[
    [
        8.234244305634809e-10,
        0.0
    ],
    [
        1.3839909617878885e-08,
        0.0
    ],
    [
        2.2615863441670862e-10,
        0.0
    ],
    [
        5.692310329990123e-09,
        0.0
    ]
]
Average error on spinq:
[
    [
        1.2143299326635026e-11,
        0.0
    ],
    [
        5.1038785434852146e-12,
        0.0
    ],
    [
        9.139690506308432e-12,
        0.0
    ],
    [
        1.6179114306580753e-11,
        0.0
    ]
]
Average error on spinqq:
[
    [
        0.02236294828

In [72]:
if not quiet:
	if passed:
		print(f"{CYAN}Overall{STANDARD}: {GREEN}Passed{STANDARD}")
	else:
		print(f"Overall: {RED}Failed{STANDARD}")
	print ("")
	print (f"{CYAN}Detailed results{STANDARD}:")
	for p in passwdProjects:
		if passwdProjects[p]:
			print(f"\t{MAGENTA}"+p+f"{STANDARD}"+f": {GREEN}Passed{STANDARD}")
		else:
			print(f"\t{MAGENTA}"+p+f"{STANDARD}"+f": {RED}Failed{STANDARD}")

Overall: Passed

Detailed results:
	pennylane: Passed
	qiskitrot: Passed
	quest: Passed
	spinq: Passed
	spinqq: Passed
	bmqsim_hw: Passed
	bmqsim: Passed
	bmqsim_alveo: Passed
